# 循环神经网络(RNN)基础实现与应用

## 教程概览

本教程全面介绍循环神经网络(RNN)的核心概念、实现原理和实际应用：

### 1. RNN基础实现
- 从零开始实现RNN的前向传播
- 理解隐藏状态的更新机制
- 掌握RNN的核心数学公式

### 2. Keras/TensorFlow实现
- 使用高层API快速构建RNN模型
- 理解`return_sequences`参数的作用
- 构建深层RNN架构

### 3. 实战：情感分析
- IMDB电影评论情感分类
- 对比SimpleRNN与LSTM的性能
- 可视化训练过程和模型评估

### 学习目标
- 掌握RNN的工作原理和计算流程
- 学会使用Keras构建各种RNN模型
- 理解RNN在序列任务中的应用
- 认识SimpleRNN的局限性和LSTM的优势

In [ ]:
"""
循环神经网络(RNN)的基础实现
演示RNN的核心计算流程和状态传递机制
"""
import numpy as np

# 设置随机种子以保证结果可复现
RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)

# 定义网络参数
timesteps = 100         # 时间序列长度
input_features = 32     # 每个时间步的输入特征维度
output_features = 64    # 隐藏状态和输出的特征维度

# 生成模拟的输入序列数据
# 形状为 (timesteps, input_features)
inputs = np.random.rand(timesteps, input_features)

# 初始化隐藏状态为零向量
state_t = np.zeros((output_features,))

# 初始化权重矩阵
# W: 输入到隐藏状态的权重矩阵
# U: 前一时刻隐藏状态到当前隐藏状态的权重矩阵
# b: 偏置向量
W = np.random.random((output_features, input_features))
U = np.random.random((output_features, output_features))
b = np.random.random((output_features,))

# 按时间步迭代计算RNN的输出
successive_outputs = []
for t in range(timesteps):
    # RNN核心公式: h_t = tanh(W * x_t + U * h_{t-1} + b)
    # 其中 x_t 是当前时刻的输入，h_{t-1} 是上一时刻的隐藏状态
    output_t = np.tanh(np.dot(W, inputs[t]) + np.dot(U, state_t) + b)
    
    # 保存当前时刻的输出
    successive_outputs.append(output_t)
    
    # 更新隐藏状态，用于下一时刻的计算
    state_t = output_t

# 将所有时间步的输出堆叠成三维张量
# 形状为 (timesteps, output_features)
final_output = np.stack(successive_outputs, axis=0)
print(f"输出形状: {final_output.shape}")
print(f"期望形状: ({timesteps}, {output_features})")

### RNN的计算机制详解

RNN的核心思想是在处理序列数据时保持一个**隐藏状态(hidden state)**，该状态在每个时间步都会被更新，并影响后续时间步的计算。

**关键要素：**
1. **输入权重矩阵 W**: 将当前时刻的输入转换为隐藏状态空间
2. **循环权重矩阵 U**: 将上一时刻的隐藏状态传递到当前时刻
3. **偏置向量 b**: 为网络提供额外的学习自由度
4. **激活函数**: 通常使用tanh或ReLU引入非线性

**信息流动路径：**
- 当前输入 → 通过W加权 → 
- 上一时刻状态 → 通过U加权 → 
- 两者相加加上偏置 → 激活函数 → 当前时刻的隐藏状态

## RNN的数学表达式

$$h_t = \text{activation}(W \cdot x_t + U \cdot h_{t-1} + b)$$

其中：
- $h_t$: 当前时刻的隐藏状态
- $x_t$: 当前时刻的输入
- $h_{t-1}$: 上一时刻的隐藏状态
- $W$: 输入权重矩阵
- $U$: 循环权重矩阵(有时也记作$W_{rec}$)
- $b$: 偏置向量
- $\text{activation}$: 激活函数(通常为tanh)

In [ ]:
"""
使用Keras构建简单的RNN模型
演示嵌入层和SimpleRNN层的组合使用
"""
import tensorflow as tf
from keras.layers import SimpleRNN, Embedding
from keras.models import Sequential

# 构建基础RNN模型
model = Sequential([
    # 嵌入层: 将词索引转换为密集向量表示
    # 参数: (词汇表大小, 嵌入维度)
    Embedding(input_dim=10000, output_dim=32),
    
    # SimpleRNN层: 处理序列数据
    # 参数: units=32表示隐藏状态的维度
    # 默认只返回最后一个时间步的输出
    SimpleRNN(units=32)
])

model.summary()

In [ ]:
"""
演示return_sequences参数的作用
当return_sequences=True时，RNN会返回所有时间步的输出
"""
model = Sequential([
    Embedding(input_dim=10000, output_dim=32),
    
    # return_sequences=True: 返回完整的输出序列
    # 输出形状: (batch_size, timesteps, units)
    # 适用于需要序列到序列映射的任务
    SimpleRNN(units=32, return_sequences=True)
])

model.summary()

In [ ]:
"""
构建深层RNN网络
通过堆叠多个RNN层来增加模型的表达能力
"""
model = Sequential([
    Embedding(input_dim=10000, output_dim=32),
    
    # 第一层RNN: 必须设置return_sequences=True才能将序列传递给下一层
    SimpleRNN(units=32, return_sequences=True),
    
    # 中间层RNN: 同样需要return_sequences=True
    SimpleRNN(units=32, return_sequences=True),
    
    # 第三层RNN
    SimpleRNN(units=32, return_sequences=True),
    
    # 最后一层RNN: 可以只返回最后一个时间步的输出
    # 这里为了演示保持return_sequences=True
    SimpleRNN(units=32, return_sequences=True)
])

# 注意：深层RNN容易出现梯度消失问题
# 实践中通常使用LSTM或GRU来替代SimpleRNN
model.summary()

In [ ]:
"""
准备IMDB情感分析数据集
使用Keras内置的IMDB数据集进行演示
"""
import tensorflow as tf
import os
from keras.datasets import imdb
from keras.preprocessing.sequence import pad_sequences

# 数据集参数配置
MAX_FEATURES = 10000    # 词汇表大小，只保留最常见的10000个词
MAX_LEN = 500           # 序列最大长度
BATCH_SIZE = 32

print("=" * 70)
print("加载IMDB数据集")
print("=" * 70)

# 加载IMDB数据集（Keras内置）
# 数据格式: 每个样本是一个整数序列，代表单词索引
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=MAX_FEATURES)

print(f"\n训练集大小: {len(x_train)}")
print(f"测试集大小: {len(x_test)}")
print(f"标签分布 - 训练集: 负面={sum(y_train == 0)}, 正面={sum(y_train == 1)}")

# 查看第一条评论的原始数据
print(f"\n第一条评论的长度: {len(x_train[0])}")
print(f"第一条评论的前20个词索引: {x_train[0][:20]}")
print(f"对应标签: {y_train[0]} ({'正面' if y_train[0] == 1 else '负面'})")

print("\n" + "=" * 70)
print("对序列进行填充和截断")
print("=" * 70)

# 将序列填充到相同长度
# padding='post': 在序列末尾填充
# truncating='post': 从序列末尾截断
x_train = pad_sequences(x_train, maxlen=MAX_LEN, padding='post', truncating='post')
x_test = pad_sequences(x_test, maxlen=MAX_LEN, padding='post', truncating='post')

print(f"填充后的训练集形状: {x_train.shape}")
print(f"填充后的测试集形状: {x_test.shape}")

# 创建验证集
VALIDATION_SPLIT = 0.2
validation_samples = int(VALIDATION_SPLIT * len(x_train))

x_val = x_train[:validation_samples]
y_val = y_train[:validation_samples]
x_train_final = x_train[validation_samples:]
y_train_final = y_train[validation_samples:]

print(f"\n最终数据划分:")
print(f"  训练集: {len(x_train_final)} 样本")
print(f"  验证集: {len(x_val)} 样本")
print(f"  测试集: {len(x_test)} 样本")

print("\n数据准备完成！")

In [ ]:
"""
构建并训练基于SimpleRNN的情感分析模型
演示双层RNN架构在文本分类任务中的应用
"""
from keras.layers import Dense, Embedding, SimpleRNN
from keras.models import Sequential
from keras.optimizers import Adam

print("=" * 70)
print("构建SimpleRNN模型")
print("=" * 70)

# 构建模型架构
model = Sequential([
    # 嵌入层: 将词索引映射到稠密向量空间
    Embedding(input_dim=MAX_FEATURES, output_dim=32),
    
    # 第一层RNN: 返回完整序列以便堆叠
    SimpleRNN(units=32, return_sequences=True, dropout=0.2),
    
    # 第二层RNN: 只返回最后时间步的输出
    SimpleRNN(units=32, dropout=0.2),
    
    # 输出层: 二分类任务
    Dense(1, activation='sigmoid')
])

# 编译模型
model.compile(
    optimizer=Adam(learning_rate=0.001),
    loss='binary_crossentropy',
    metrics=['accuracy']
)

model.summary()

print("\n" + "=" * 70)
print("开始训练（使用少量epoch进行快速测试）")
print("=" * 70)

# 训练模型
# 注意：为了快速测试，这里只训练3个epoch
# 实际应用中可能需要更多epoch
history = model.fit(
    x_train_final, 
    y_train_final,
    epochs=3,                    # 测试时使用较少epoch
    batch_size=128,
    validation_data=(x_val, y_val),
    verbose=1
)

print("\n训练完成！")
print(f"最终训练准确率: {history.history['accuracy'][-1]:.4f}")
print(f"最终验证准确率: {history.history['val_accuracy'][-1]:.4f}")

In [ ]:
"""
可视化训练过程
绘制训练和验证的准确率及损失曲线
"""
import matplotlib.pyplot as plt

# 提取训练历史数据
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs_range = range(1, len(acc) + 1)

# 创建图表
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

# 绘制准确率曲线
ax1.plot(epochs_range, acc, 'bo-', label='训练准确率', linewidth=2, markersize=8)
ax1.plot(epochs_range, val_acc, 'rs-', label='验证准确率', linewidth=2, markersize=8)
ax1.set_title('训练和验证准确率', fontsize=14, fontweight='bold')
ax1.set_xlabel('Epoch', fontsize=12)
ax1.set_ylabel('准确率', fontsize=12)
ax1.legend(fontsize=11)
ax1.grid(True, alpha=0.3)

# 绘制损失曲线
ax2.plot(epochs_range, loss, 'bo-', label='训练损失', linewidth=2, markersize=8)
ax2.plot(epochs_range, val_loss, 'rs-', label='验证损失', linewidth=2, markersize=8)
ax2.set_title('训练和验证损失', fontsize=14, fontweight='bold')
ax2.set_xlabel('Epoch', fontsize=12)
ax2.set_ylabel('损失', fontsize=12)
ax2.legend(fontsize=11)
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# 分析过拟合情况
if len(acc) > 1:
    train_val_gap = acc[-1] - val_acc[-1]
    if train_val_gap > 0.1:
        print(f"\n⚠️  检测到过拟合: 训练准确率比验证准确率高 {train_val_gap:.2%}")
        print("建议: 增加dropout、减少模型复杂度或使用更多训练数据")
    else:
        print(f"\n✓ 模型泛化良好: 训练-验证准确率差距为 {train_val_gap:.2%}")

In [ ]:
"""
使用LSTM改进模型性能
LSTM通过门控机制解决RNN的梯度消失问题
"""
from keras.layers import LSTM

print("=" * 70)
print("构建LSTM模型")
print("=" * 70)

# 构建LSTM模型
lstm_model = Sequential([
    # 嵌入层
    Embedding(input_dim=MAX_FEATURES, output_dim=32),
    
    # LSTM层: 比SimpleRNN更善于捕捉长期依赖
    # LSTM包含三个门: 遗忘门、输入门、输出门
    LSTM(units=32, dropout=0.2, recurrent_dropout=0.2),
    
    # 输出层
    Dense(1, activation='sigmoid')
])

# 编译模型
lstm_model.compile(
    optimizer=Adam(learning_rate=0.001),
    loss='binary_crossentropy',
    metrics=['accuracy']
)

lstm_model.summary()

print("\n" + "=" * 70)
print("训练LSTM模型")
print("=" * 70)

# 训练LSTM模型
lstm_history = lstm_model.fit(
    x_train_final,
    y_train_final,
    epochs=3,                    # 测试时使用较少epoch
    batch_size=128,
    validation_data=(x_val, y_val),
    verbose=1
)

print("\n训练完成！")
print(f"最终训练准确率: {lstm_history.history['accuracy'][-1]:.4f}")
print(f"最终验证准确率: {lstm_history.history['val_accuracy'][-1]:.4f}")

# 在测试集上评估
test_loss, test_acc = lstm_model.evaluate(x_test, y_test, verbose=0)
print(f"\n测试集准确率: {test_acc:.4f}")

In [8]:
model.summary()

Model: "sequential_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_4 (Embedding)         │ (None, 500, 32)        │       320,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ (None, 32)             │         8,320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 985,061 (3.76 MB)

 Trainable params: 328,353 (1.25 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 656,708 (2.51 MB)